Follow the steps in https://docs.microsoft.com/en-us/azure/machine-learning/how-to-deploy-managed-online-endpoints to deploy the scoring function to Azure.
This scoring is on demand, i.e. it only loads the model when the client requests.

In [1]:
import urllib.request
import requests
import pandas as pd
import json
for file_name in ['test_data_1000_dominicks.csv','test_data_1002_tropicana.csv']:
    sample_data= pd.read_csv(file_name)
    sample_data.drop(['Unnamed: 0'], axis=1, inplace=True)
    #Use the below version of URL and header in case you test with remote web service (AKS)
    url ='https://many-model.westus2.inference.ml.azure.com/score'
    api_key = 'K8It9Dq12BpQu8ryt6VRXZB1AmCzfsZu' # Replace this with the API key for the web service
    headers = {'Content-Type':'application/json', 'Authorization':('Bearer '+ api_key)}



    data = {"Inputs":sample_data.to_json() }
    body = str.encode(json.dumps(data))
    resp = requests.post(url, data=body, headers=headers)
    print(resp.text)


C:\Users\janguy\Anaconda3\envs\dlresearch\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\janguy\Anaconda3\envs\dlresearch\lib\site-packages\numpy\.libs\libopenblas.NOIJJG62EMASZI6NYURL6JBKM4EVBGM7.gfortran-win_amd64.dll
C:\Users\janguy\Anaconda3\envs\dlresearch\lib\site-packages\numpy\.libs\libopenblas.PYQHXLVVQ7VESDPUVUADXEVJOBGHJPAY.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:\n%s" %


key_auth_access_denied
key_auth_access_denied


In [4]:
import urllib.request
import json
import os
import ssl
import random
def allowSelfSignedHttps(allowed):
    # bypass the server certificate verification on client side
    if allowed and not os.environ.get('PYTHONHTTPSVERIFY', '') and getattr(ssl, '_create_unverified_context', None):
        ssl._create_default_https_context = ssl._create_unverified_context

allowSelfSignedHttps(True) # this line is needed if you use self-signed certificate in your scoring service.

# Request data goes here
# The example below assumes JSON formatting which may be updated
# depending on the format your endpoint expects.
# More information can be found here:
# https://docs.microsoft.com/azure/machine-learning/how-to-deploy-advanced-entry-script
def score():
    number = random.randint(0,1000)
    data = {"data":"some_input "+str(number)}

    body = str.encode(json.dumps(data))

    url = 'http://52.137.89.247:80/api/v1/service/many-models-ray/score'
    api_key = 'lPEKXOMscRp5VZaM14hXZKOQoJTQ8VuZ' # Replace this with the API key for the web service

    # The azureml-model-deployment header will force the request to go to a specific deployment.
    # Remove this header to have the request observe the endpoint traffic rules
    headers = {'Content-Type':'application/json', 'Authorization':('Bearer '+ api_key)}

    req = urllib.request.Request(url, body, headers)

    try:
        response = urllib.request.urlopen(req)

        result = response.read()
        print(result)
    except urllib.error.HTTPError as error:
        print("The request failed with status code: " + str(error.code))

        # Print the headers - they include the requert ID and the timestamp, which are useful for debugging the failure
        print(error.info())
        print(error.read().decode("utf8", 'ignore'))
import threading
import concurrent.futures
# with concurrent.futures.ThreadPoolExecutor() as executor:
#     for i in range(20):
#         # executor.submit(score)
#         thread.start_new_thread(score,"thread "+str(i) )

for i in range(10):
    # executor.submit(score)
    threading.Thread(target = score).start()


b'"{\\"result\\": \\"deployment3 some_input 783\\"}"'


In [9]:
score()

The request failed with status code: 502
cache-control: max-age=0, private, must-revalidate
connection: close
content-length: 145
content-type: text/plain; charset=utf-8
date: Thu, 25 Aug 2022 21:45:01 GMT
server: Cowboy
x-ms-request-id: 010f6d3f-e260-4c22-8fd1-8dd36e7a1e49


Not enough working backends for service many-models-ray (deadline), last failure: timeout. 
Help: https://go.microsoft.com/fwlink/?linkid=2146748
